In [ ]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)


# Renewable generation potentials are important for constraining capacities
- Use this notebook to esimate renewable generation potentials

# General Data Sources
- International Renewable ENergy Agency (IRENA)
    - IRENA Global Atlas: https://globalatlas.irena.org/

# Hydropower Generation Potential

IRENA_tudelft_global_hydropower.csv
-----------------------------------

#### RAW DATA
https://data.4tu.nl/articles/dataset/Global_potential_hydropower_locations/12708413/1

#### CITE
Hoes O.A.C. (2014): Global potential hydropower locations. 4TU.ResearchData. Dataset. https://doi.org/10.4121/uuid:99b42e30-5a69-4a53-8e77-c954f11dbc76 Copy citation


#### IRENA LAYER (box around LAC)
"Global hydropower potential" (TU-Delft)

#### ACCESS DATE: 
2/20/2023

# Geothermal Generation Potential

IRENA_hdr_Pt_14_global.tiff (BEARDSMORE)
----------------------------------------

#### RAW DATA


#### CITE
Beardsmore et al. (2011). A Protocol for Estimating and Mapping Global EGS Potential. DOI: 10.13140/RG.2.2.33475.71204

#### IRENA LAYER (box around LAC)
Layer "Global total EGS technical potential 14% recovery" (HDR)

#### ACCESS DATE: 
2/20/2023



IRENA_hdr_TotalTheoretical_global.tiff (BEARDSMORE)
---------------------------------------------------

#### RAW DATA


#### CITE
Beardsmore et al. (2011). A Protocol for Estimating and Mapping Global EGS Potential. DOI: 10.13140/RG.2.2.33475.71204

#### IRENA LAYER (box around LAC)
Layer "Global EGS theoretical potential 2.5–3.5 km depth" (HDR)

#### ACCESS DATE: 
2/20/2023
